# Semantic Kernel နှင့် OpenBnB MCP Server ပေါင်းစပ်မှု

ဒီ notebook က Semantic Kernel ကို OpenBnB MCP server နဲ့ပေါင်းစပ်ပြီး MCPStdioPlugin ကိုသုံးကာ အစစ်အမှန် Airbnb အိမ်ခြံမြေများကို ရှာဖွေဖို့အတွက် အသုံးပြုနည်းကို ပြသထားပါတယ်။ LLM Access အတွက် Azure AI Foundry ကို အသုံးပြုထားပါတယ်။ သင့်ပတ်ဝန်းကျင် variable များကို စနစ်တကျပြင်ဆင်ရန် [Setup Lesson](/00-course-setup/README.md) ကို လိုက်နာနိုင်ပါတယ်။


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP Plugin ချိတ်ဆက်မှု ဖန်တီးခြင်း

ကျွန်တော်တို့ [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) ကို MCPStdioPlugin အသုံးပြု၍ ချိတ်ဆက်ပါမည်။ ဒီ server သည် @openbnb/mcp-server-airbnb package ကို အသုံးပြု၍ Airbnb ရှာဖွေမှု လုပ်ဆောင်နိုင်စွမ်းပေးပါသည်။


## Client ဖန်တီးခြင်း

ဤနမူနာတွင် LLM ကို အသုံးပြုရန်အတွက် Azure AI Foundry ကို အသုံးပြုမည်ဖြစ်သည်။ သင့်ပတ်ဝန်းကျင် အပြောင်းအလဲများကို မှန်ကန်စွာ စီစဉ်ထားရှိရန် သေချာပါ။


## ပတ်ဝန်းကျင်ဖွဲ့စည်းမှု

Azure OpenAI အပြင်အဆင်များကို ဖွဲ့စည်းပါ။ အောက်ပါပတ်ဝန်းကျင်အပြောင်းအလဲများကို သတ်မှတ်ထားရှိရန် သေချာပါစေ-
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP အစိတ်အပိုင်း ပေါင်းစည်းမှုကို နားလည်ခြင်း

ဒီ notebook က **အမှန်တကယ် OpenBnB MCP server** ကို ချိတ်ဆက်ပြီး Airbnb ရှာဖွေမှုလုပ်ဆောင်ချက်ကို ပေးစွမ်းပါတယ်။

### ဘယ်လိုအလုပ်လုပ်သလဲ:

1. **MCPStdioPlugin**: MCP server နဲ့ standard input/output ဆက်သွယ်မှုကို အသုံးပြုပါတယ်
2. **Real NPM Package**: `@openbnb/mcp-server-airbnb` ကို npx မှတဆင့် download လုပ်ပြီး run လုပ်ပါတယ်
3. **Live Data**: Airbnb API များမှ အမှန်တကယ် property data ကို ပြန်ပေးပါတယ်
4. **Function Discovery**: MCP server မှ ရရှိနိုင်သော function များကို agent က အလိုအလျောက် ရှာဖွေပါတယ်

### ရရှိနိုင်သော Function များ:

OpenBnB MCP server က အောက်ပါ function များကို ပုံမှန်ပေးပါတယ်။
- **search_listings** - တည်နေရာနှင့် criteria အတိုင်း Airbnb property များကို ရှာဖွေပါ
- **get_listing_details** - အထူး property များအကြောင်း အသေးစိတ်အချက်အလက်ရယူပါ
- **check_availability** - သတ်မှတ်ရက်များအတွက် ရရှိနိုင်မှုကို စစ်ဆေးပါ
- **get_reviews** - Property များအတွက် ပြန်လည်သုံးသပ်ချက်များကို ရယူပါ
- **get_host_info** - Property host များအကြောင်း အချက်အလက်ရယူပါ

### လိုအပ်ချက်များ:

- **Node.js** ကို သင့်စနစ်တွင် install လုပ်ထားရမည်
- MCP server package ကို download လုပ်ရန် **Internet connection** လိုအပ်သည်
- **NPX** ရရှိနိုင်ရမည် (Node.js နှင့်အတူ ပါဝင်သည်)

### ချိတ်ဆက်မှုကို စမ်းသပ်ခြင်း:

MCP server ကို လက်တွေ့စမ်းသပ်ရန် အောက်ပါ command ကို run လုပ်နိုင်ပါသည်။
```bash
npx -y @openbnb/mcp-server-airbnb
```

ဒီ command က OpenBnB MCP server ကို download လုပ်ပြီး start လုပ်ပါမည်၊ ထို့နောက် Semantic Kernel က အမှန်တကယ် Airbnb data ရရှိရန် ချိတ်ဆက်ပါမည်။


## OpenBnB MCP Server နှင့် Agent ကို အလုပ်လုပ်စေခြင်း

အခုတော့ OpenBnB MCP server နှင့် ချိတ်ဆက်ထားသော AI Agent ကို အလုပ်လုပ်စေပါမယ်။ ဒီ Agent ကတော့ စတော့ခေါ့မြို့မှာ လူကြီး ၂ ယောက်နဲ့ ကလေး ၁ ယောက်အတွက် အမှန်တကယ်ရှိတဲ့ Airbnb အိမ်ခန်းတွေကို ရှာဖွေမှာ ဖြစ်ပါတယ်။ `user_inputs` စာရင်းကို ပြင်ဆင်ပြီး ရှာဖွေမှု အချက်အလက်တွေကို ပြောင်းလဲနိုင်ပါတယ်။


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# အကျဉ်းချုပ်
ဂုဏ်ပြုပါတယ်! သင်သည် Model Context Protocol (MCP) ကို အသုံးပြု၍ အိမ်ရာရှာဖွေမှုနှင့် ပေါင်းစပ်ထားသော AI အေးဂျင့်တစ်ခုကို အောင်မြင်စွာ တည်ဆောက်နိုင်ခဲ့ပါပြီ။

## အသုံးပြုထားသော နည်းပညာများ:
- Semantic Kernel - Azure OpenAI ဖြင့် ဉာဏ်ရည်ရှိသော အေးဂျင့်များ တည်ဆောက်ရန်
- Azure AI Foundry - LLM စွမ်းရည်နှင့် စကားပြောအပြီးသတ်မှုအတွက်
- MCP (Model Context Protocol) - စံပြပစ္စည်းများ ပေါင်းစပ်မှုအတွက်
- OpenBnB MCP Server - အမှန်တကယ် Airbnb ရှာဖွေမှု လုပ်ဆောင်နိုင်စွမ်းအတွက်
- Node.js/NPX - အပြင်ဘက် MCP server ကို အလုပ်လုပ်စေရန်

## သင်လေ့လာခဲ့သောအရာများ:
- MCP ပေါင်းစပ်မှု: Semantic Kernel အေးဂျင့်များကို အပြင်ဘက် MCP server များနှင့် ချိတ်ဆက်ခြင်း
- အချိန်နှင့်တပြေးညီ ဒေတာရယူမှု: အမှန်တကယ် Airbnb အိမ်ခြံမြေများကို API များမှ ရှာဖွေခြင်း
- Protocol ဆက်သွယ်မှု: အေးဂျင့်နှင့် MCP server အကြား stdio ဆက်သွယ်မှုကို အသုံးပြုခြင်း
- လုပ်ဆောင်မှု ရှာဖွေမှု: MCP server များမှ ရရှိနိုင်သော လုပ်ဆောင်မှုများကို အလိုအလျောက် ရှာဖွေခြင်း
- Streaming Responses: လုပ်ဆောင်မှု ခေါ်ဆိုမှုများကို အချိန်နှင့်တပြေးညီ ဖမ်းယူပြီး မှတ်တမ်းတင်ခြင်း
- HTML Rendering: အေးဂျင့်၏ အဖြေများကို စတိုင်ပြုလုပ်ထားသော ဇယားများနှင့် အပြန်အလှန် ပြသမှုများဖြင့် ဖော်ပြခြင်း

## နောက်တစ်ဆင့်များ:
- ထပ်မံ MCP server များ (ရာသီဥတု, လေယာဉ်, အစားအစာဆိုင်များ) ပေါင်းစပ်ပါ
- MCP နှင့် A2A protocols ပေါင်းစပ်ထားသော အေးဂျင့်စနစ်များ တည်ဆောက်ပါ
- သင်၏ကိုယ်ပိုင် ဒေတာရင်းမြစ်များအတွက် အထူးပြု MCP server များ ဖန်တီးပါ
- အစည်းအဝေးများအကြား ဆွေးနွေးမှုမှတ်ဉာဏ်ကို အမြဲတမ်း ထည့်သွင်းပါ
- Azure Functions တွင် MCP server စီမံခန့်ခွဲမှုဖြင့် အေးဂျင့်ကို တင်သွင်းပါ
- အသုံးပြုသူ အတည်ပြုမှုနှင့် အိမ်ရာဘွတ်ကင်စွမ်းရည်များ ထည့်သွင်းပါ



---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်ဆောင်ရွက်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပါယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။ 
